In [ ]:
import timm
import torch
from PIL import Image
import numpy as np

# %load "../segmentation.py"
import numpy as np
import cv2
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch

from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset

# Imports segmenter for getting ROI from frames
import sys
sys.path.append('../')
from segmenter import crop_with_hand_entropy_seg

torch.manual_seed(0)

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

# Device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = timm.create_model("vit_base_patch32_224", num_classes=116)

In [ ]:
trained_model = torch.load("../models/vit_base_patch32_224.pt", map_location=torch.device("cpu"))

In [ ]:
model.load_state_dict(trained_model["model_state_dict"])

In [ ]:
with open("../labels.txt") as f:
    prod = []
    for line in f.readlines():
        prod.append(line)

In [ ]:
def infer(image_path,model):
   img = Image.open(image_path)
   mean = [0.485, 0.456, 0.406] 
   std = [0.229, 0.224, 0.225]
   test_transforms = timm.data.create_transform(
      input_size=224, mean=mean, std=std
   )
   img_normalized = test_transforms(img).float()
   img_normalized = img_normalized.unsqueeze_(0)
   img_normalized = img_normalized.to("cpu")
   with torch.no_grad():
      model.eval()  
      output =model(img_normalized)
      index = output.data.cpu().numpy().argmax()
      return prod[index].strip(), output

def infer_video(frame,model):
   frame = frame[256:812,512:1216,]
   img = Image.fromarray(np.uint8(frame)).convert('RGB')
   mean = [0.485, 0.456, 0.406] 
   std = [0.229, 0.224, 0.225]
   test_transforms = timm.data.create_transform(
      input_size=224, mean=mean, std=std
   )
   img_normalized = test_transforms(img).float()
   img_normalized = img_normalized.unsqueeze_(0)
   img_normalized = img_normalized.to("cpu")
   with torch.no_grad():
      model.eval()  
      output =model(img_normalized)
      return output.max(), frame, frame.mean()

def get_ratio(image):
    h, w, _ = image.shape
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV)[1]

    pixels = cv2.countNonZero(thresh)
    ratio = (pixels/(h * w)) * 100
    return ratio

In [ ]:
def image_colorfulness(image):
	# split the image into its respective RGB components
	(B, G, R) = cv2.split(image.astype("float"))
	# compute rg = R - G
	rg = np.absolute(R - G)
	# compute yb = 0.5 * (R + G) - B
	yb = np.absolute(0.5 * (R + G) - B)
	# compute the mean and standard deviation of both `rg` and `yb`
	(rbMean, rbStd) = (np.mean(rg), np.std(rg))
	(ybMean, ybStd) = (np.mean(yb), np.std(yb))
	# combine the mean and standard deviations
	stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
	meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
	# derive the "colorfulness" metric and return it
	return stdRoot + (0.3 * meanRoot)

In [ ]:
# Automatic brightness and contrast optimization with optional histogram clipping
def automatic_brightness_and_contrast(image, clip_hist_percent=1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate grayscale histogram
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)
    
    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))
    
    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0
    
    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    
    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    
    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha
    
    '''
    # Calculate new histogram with desired range and show histogram 
    new_hist = cv2.calcHist([gray],[0],None,[256],[minimum_gray,maximum_gray])
    plt.plot(hist)
    plt.plot(new_hist)
    plt.xlim([0,256])
    plt.show()
    '''

    auto_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return auto_result

In [ ]:
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt 

video_location = "../test-videos/"
videos = ["testA_1.mp4"]
images_list = []
ratios_list = []
colors_list = []
metrics_list = []

for video in videos:
  vidcap = cv2.VideoCapture(video_location+video)
  fps = vidcap.get(cv2.CAP_PROP_FPS)      
  frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
  resized_image = []

  for i in tqdm(range(frame_count)):
    try:
      success,image = vidcap.read()
      image = image[256:896, 512:1400]
      image = automatic_brightness_and_contrast(image)
      image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
      resized_image.append(image)
    except:
      pass
  ratios = []
  colors = []
  metrics = []
  for img in resized_image:
    # ratio = get_ratio(img)
    color = image_colorfulness(img)
    # metric = color*color*ratio
    colors.append(color)
    # ratios.append(ratio)
    # metrics.append(metric)
  
  images_list.append(resized_image)
  # ratios_list.append(ratios)
  colors_list.append(colors)
  # metrics_list.append(metrics)
  # print(video)
  # plt.plot(metrics)
  # plt.show()
  # plt.plot(ratios)
  # plt.show()
  plt.plot(colors)
  plt.show()

In [ ]:
import scipy.fftpack as fftpack

def get_sharpness(image):
    im = Image.fromarray(image).convert('L') # to grayscale
    array = np.asarray(im, dtype=np.int32)

    gy, gx = np.gradient(array)
    gnorm = np.sqrt(gx**2 + gy**2)
    sharpness = np.average(gnorm)
    return sharpness

def smooth_data_fft(arr, span):  # the scaling of "span" is open to suggestions
    w = fftpack.rfft(arr)
    spectrum = w ** 2
    cutoff_idx = spectrum < (spectrum.max() * (1 - np.exp(-span / 2000)))
    w[cutoff_idx] = 0
    return fftpack.irfft(w)

In [ ]:
from scipy.signal import savgol_filter, argrelextrema,find_peaks
import math
images_with_object = []
idx = []
count = 1
for images, colors in zip(images_list, colors_list):
    frames_with_object = []
    rolling_colors = smooth_data_fft(colors, 20)
    maximas = find_peaks(rolling_colors)
    for maxima in maximas[0]:
        candidate_frame = maxima
        max_sharpness = -1
        max_metric = -1
        for i in range(-21,22,7):
            current_frame = maxima+i
            if(current_frame>=len(images)):
                continue
            sharpness = get_sharpness(images[current_frame])
            colorfulness = image_colorfulness(images[current_frame])
            ratio = get_ratio(images[current_frame])
            metric = colorfulness*colorfulness * ratio
            metric = math.sqrt(metric)
            if(sharpness>max_sharpness and metric>max_metric):
                max_sharpness = sharpness
                max_metric = metric
                candidate_frame = current_frame
        if(max_metric>110):
            frames_with_object.append((images[candidate_frame],candidate_frame))
            idx.append(candidate_frame)
            count+=1
            plt.imshow(images[candidate_frame][:,:,[2,1,0]])
            plt.show()

            print(f"Color {colorfulness}")
            print(f"Sharp {sharpness}")
            print(f"Metric {max_metric}")

    plt.plot(rolling_colors)
    plt.show()
    print("NEXT")
    images_with_object.append(frames_with_object)

In [ ]:
idx

In [ ]:
plt.figure(figsize=(12,3))
plt.subplot(131)
plt.plot(colors)
plt.xlabel("Frames Count")
plt.ylabel("Colorfulness Value")
plt.title("(a) Original Colorfulness Value Array")
plt.subplot(132)
plt.plot(rolling_colors)
plt.xlabel("Frames Count")
plt.ylabel("Colorfulness Value")
plt.title("(b) Filtered Colorfulness Value Array (FFT)")
plt.subplot(133)
plt.plot(rolling_colors,"-D", markevery=idx)
plt.xlabel("Frames Count")
plt.ylabel("Colorfulness Value")
plt.title("(c) Selected Frames (dots)")
plt.tight_layout()
plt.savefig("colorfulness-fft.png",dpi=300)
plt.show()

In [ ]:
def get_blobs(image):
    img = np.array(image)

    # Convert you image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Threshold the image to extract only objects that are not black
    # You need to use a one channel image, that's why the slice to get the first layer
    tv, thresh = cv2.threshold(gray[:,:,0], 1, 255, cv2.THRESH_BINARY)

    # Get the contours from your thresholded image
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    # Create a copy of the original image to display the output rectangles
    output = img.copy()

    # Loop through your contours calculating the bounding rectangles and plotting them
    # for c in contours:
    #     x, y, w, h = cv2.boundingRect(c)
        # if(w*h>(224*244)/10):
    c = max(contours, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    return x,y,w,h
    # cv2.rectangle(output, (x,y), (x+w, y+h), (0, 0, 255), 2)
    # # Display the output image
    # plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    # plt.show()

In [ ]:

transforms_image = transforms.Compose([transforms.Resize((224, 224)),
                                          transforms.CenterCrop((224, 224)),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])



def preprocess_image_classification(frame):
   img = Image.fromarray(frame).convert('RGB')

   mean = [0.4124, 0.3856, 0.3493] 
   std = [0.2798, 0.2703, 0.2726]
   test_transforms = timm.data.create_transform(
      input_size=224, mean=mean, std=std
   )
   img_normalized = test_transforms(img).float()
   return img_normalized

def infer_frame(frame, model):
   img = Image.fromarray(frame).convert('RGB')
   img = transforms_image(img)
   segmented_image = crop_with_hand_entropy_seg(img)["roi"]
   x,y,w,h = get_blobs(segmented_image)
   image_roi = frame[y:y+h, x:x+w]
   plt.imshow(image_roi[:,:,[2,1,0]])
   plt.show()
   img_normalized = preprocess_image_classification(frame=image_roi[:,:,[2,1,0]])
   img_normalized = img_normalized.unsqueeze_(0)
   img_normalized = img_normalized.to("cpu")
   with torch.no_grad():
      model.eval()
      output =model(img_normalized)
      index = output.data.cpu().numpy().argmax()
      op_array = output.data.cpu().numpy()
      print(op_array[0][index])
      return index+1


In [ ]:

video_id = []
object_id = []
timestamp = []
for idx in range(len(images_with_object)):
    lastoutput = ""
    for image,frame in images_with_object[idx]:
        output = infer_frame(image, model)
        lastoutput=output
        video_id.append(idx+1)
        object_id.append(output)
        print(prod[output-1])
        timestamp.append(int(frame/60))    

In [ ]:
with open('vit_duplication_removed.txt', 'w') as f:

    prev_vid = -1
    prev_obj = -1
    for vid, obj, tim in zip(video_id, object_id, timestamp):
        if(vid==prev_vid and obj==prev_obj):
            continue
        f.write(f'{vid} {obj} {tim}\n')
        prev_vid = vid
        prev_obj = obj
    f.close()